In [ ]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
import jieba
import numpy as np

# 加載模型和分詞器
model_name = "ckiplab/bert-base-chinese-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# 設置設備（使用 GPU 若可用，包括 CUDA 和 Apple MPS）
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("使用 CUDA GPU 加速")
elif hasattr(torch, 'backends') and hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    print("使用 Apple Metal (MPS) 加速")
else:
    device = torch.device("cpu")
    print("使用 CPU 運算")

model.to(device)
model.eval()  # 設置模型為評估模式

# 讀取文本文件
with open('./data/moneynews20250313.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 分詞並記錄位置
words = list(jieba.cut(text))
word_list = []
start = 0
for word in words:
    word_start = text.find(word, start)
    word_end = word_start + len(word)
    word_list.append((word, word_start, word_end))
    start = word_end

# 定義獲取單詞標籤的函數
def get_entity_type(word_start, word_end, offset_mapping, predicted_labels, label_names):
    """
    獲取單詞的實體類型
    """
    word_token_indices = []
    
    # 找出覆蓋這個詞的所有標記
    for i, (token_start, token_end) in enumerate(offset_mapping):
        if token_end > word_start and token_start < word_end:
            word_token_indices.append(i)
    
    if not word_token_indices:
        return 'O'
    
    # 獲取每個標記的標籤
    token_labels = [label_names[predicted_labels[i]] for i in word_token_indices]
    
    # 檢查是否有B-開頭的標籤
    b_labels = [label for label in token_labels if label.startswith('B-')]
    if b_labels:
        # 如果有B-開頭的標籤，返回其實體類型
        entity_type = b_labels[0].split('-')[1]
        return entity_type
    
    # 檢查是否有I-開頭的標籤
    i_labels = [label for label in token_labels if label.startswith('I-')]
    if i_labels:
        # 如果有I-開頭的標籤，返回其實體類型
        entity_type = i_labels[0].split('-')[1]
        return entity_type
    
    # 如果沒有B-或I-開頭的標籤，返回O
    return 'O'

# 分塊處理長文本
def process_text_in_chunks(text, word_list, max_length=480):  # 設置小於512的最大長度，留出一些餘量
    results = []  # 存儲每個詞的標籤
    
    # 按字符位置將文本分成多個塊
    chunks = []
    current_chunk_start = 0
    current_chunk_words = []
    
    for word, start, end in word_list:
        # 如果這個詞會使當前塊超過最大長度，則開始新塊
        if end - current_chunk_start > max_length and current_chunk_words:
            chunks.append((current_chunk_start, current_chunk_words))
            current_chunk_start = start
            current_chunk_words = []
        
        # 將詞添加到當前塊
        current_chunk_words.append((word, start, end))
    
    # 添加最後一個塊
    if current_chunk_words:
        chunks.append((current_chunk_start, current_chunk_words))
    
    # 處理每個塊
    for chunk_start, chunk_words in chunks:
        # 提取塊的文本
        if len(chunk_words) == 0:
            continue
            
        chunk_end = chunk_words[-1][2]
        chunk_text = text[chunk_start:chunk_end]
        
        # 標記化文本
        encoding = tokenizer(chunk_text, return_tensors="pt", return_offsets_mapping=True)
        offset_mapping = encoding.pop("offset_mapping")[0]  # 取出 offset_mapping 並保存
        
        # 將輸入移至設備
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        
        # 進行推理
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # 獲取預測標籤
        predicted_labels = outputs.logits[0].argmax(dim=-1).cpu().numpy()
        label_names = [model.config.id2label[i] for i in range(len(model.config.id2label))]
        
        # 獲取每個單詞的標籤
        for word, start, end in chunk_words:
            # 需要調整在塊內的相對位置
            relative_start = start - chunk_start
            relative_end = end - chunk_start
            label = get_entity_type(relative_start, relative_end, offset_mapping, predicted_labels, label_names)
            results.append((word, label))
        
        print(f"已處理 {len(chunk_words)} 個詞...")
    
    return results

# 處理文本並獲取結果
all_pairs = process_text_in_chunks(text, word_list)

# 定義處理句子的函數
def format_sentence_output(sentence, pairs):
    """
    將一個句子的標籤結果按指定格式輸出，僅列出 ORG, PER, LOC 標籤
    """
    # result = f"來源：{sentence}\n輸出：\n"
    result = f"\n"
    for word, label in pairs:
        # 僅顯示 ORG, PER, LOC 標籤，其他都標為 O
        if label in ['ORG', 'PER', 'LOC']:
            result += f"{word} - {label}\n"
        else:
            result += f"{word} - O\n"
    return result

# 將結果按句子分組
sentences = []
current_sentence = ""
current_pairs = []

for word, label in all_pairs:
    current_sentence += word
    current_pairs.append((word, label))
    
    # 使用標點符號來判斷句子結束
    if word in ["。", "！", "？", "…", "\n"]:
        if current_sentence.strip():  # 確保句子不為空
            sentences.append((current_sentence, current_pairs))
        current_sentence = ""
        current_pairs = []

# 處理最後一個句子（如果有）
if current_sentence.strip():
    sentences.append((current_sentence, current_pairs))

# 輸出結果
output = ""
for sentence, pairs in sentences:
    formatted_output = format_sentence_output(sentence, pairs)
    # 移除多餘的換行，使格式符合要求
    formatted_output = formatted_output.replace("\n輸出：\n", "")
    output += formatted_output + "\n"
    
# 打印前幾個句子作為示例
print(output[:1000] + "...\n")

# 將結果寫入文件
with open('./output_ner_results_formatted.txt', 'w', encoding='utf-8') as f:
    f.write(output)

print(f"\n處理完成！共識別了 {len(all_pairs)} 個詞，分成 {len(sentences)} 個句子。")
print(f"結果已保存至 output_ner_results_formatted.txt")
print("注意：已按要求僅保留 ORG, PER, LOC 標籤，其他標籤均轉為 O")

# 測試樣例
test_text = "企業在AI應用落地策略的推展不應僅視為單獨專案執行。"
print("\n測試樣例：")
test_words = list(jieba.cut(test_text))
test_word_list = []
start = 0
for word in test_words:
    word_start = test_text.find(word, start)
    word_end = word_start + len(word)
    test_word_list.append((word, word_start, word_end))
    start = word_end

test_results = process_text_in_chunks(test_text, test_word_list)
print(format_sentence_output(test_text, test_results))

使用 CPU 運算
已處理 274 個詞...
已處理 314 個詞...
已處理 317 個詞...
已處理 322 個詞...
已處理 328 個詞...
已處理 326 個詞...
已處理 317 個詞...
已處理 315 個詞...
已處理 297 個詞...
已處理 321 個詞...
已處理 326 個詞...
已處理 322 個詞...
已處理 317 個詞...
已處理 318 個詞...
已處理 328 個詞...
已處理 325 個詞...
已處理 313 個詞...
已處理 321 個詞...
已處理 315 個詞...
已處理 308 個詞...
已處理 333 個詞...
已處理 308 個詞...
已處理 322 個詞...
已處理 328 個詞...
已處理 303 個詞...
已處理 316 個詞...
已處理 317 個詞...
已處理 319 個詞...
已處理 318 個詞...
已處理 311 個詞...
已處理 310 個詞...
已處理 318 個詞...
已處理 318 個詞...
已處理 305 個詞...
已處理 309 個詞...
已處理 312 個詞...
已處理 326 個詞...
已處理 328 個詞...
已處理 308 個詞...
已處理 320 個詞...
已處理 311 個詞...
已處理 316 個詞...
已處理 326 個詞...
已處理 318 個詞...
已處理 303 個詞...
已處理 312 個詞...
已處理 308 個詞...
已處理 308 個詞...
已處理 315 個詞...
已處理 313 個詞...
已處理 294 個詞...
已處理 323 個詞...
已處理 312 個詞...
已處理 320 個詞...
已處理 321 個詞...
已處理 316 個詞...
已處理 323 個詞...
已處理 319 個詞...
已處理 18 個詞...

第一章 - O
　 - O
滅門 - O

 - O


________________________________________ - O

 - O


　 - O
　 - O
和 - O
風熏 - O
柳 - O
， - O
花香 - O
醉人 - O
， - O
正是 - O
南國 - O
春光 - O
漫爛 -